Figure out how to create evaluation data for 5 operations: multiply 1-digit, carry, mod 10, sum, concat; Share their plan on Slack with us before they run it.
Design* prompts for LLaMA2 and/or FLAN-T5 to answer instances of 5 operations, and share them with us. * Following prior work: they will check what others have done by querying https://aclanthology.org/.
Use prompts to get answers.

Report accuracies for each of the 5 operations.

```
> Overall goal:
  Input:
  "x * y =" st x,y in Z+
  Output:
  "z" st z is the product of x and y

> Suboperations:

  Multiply 1-digit:
    Input:
    "x * y =" st x,y in [0-9]
    Output:
    "z" st z is the product of x and y

  Carry:
    Input:
    "x // 10 =" st x in [0-99]
    Output:
    "z" st z is the digit that is in the tens place of x

  Sum:
    Input:
    "x + y =" st x,y in [0-99]
    Output:
    "z" st that z is the sum of x and y
    
  Concatenate:
    Input:
    "a, b, c =" st a, b, c in Z
    Output:
    "abc"
```



In [1]:
with open("multiply.csv", "w") as file:
  for x in range(0, 1000):
    for y in range(0, 1000):
      file.write(f'"Multiply two numbers.\n###\n5 * 3 = 15\n###\n{x} * {y} =",{x * y}\n')

In [2]:
with open("multiply_1_digit.csv", "w") as file:
  for x in range(0, 10):
    for y in range(0, 10):
      file.write(f'"Multiply two numbers.\n###\n5 * 3 = 15\n###\n{x} * {y} =",{x * y}\n')

In [3]:
with open("carry.csv", "w") as file:
  for x in range(0, 100):
    file.write(f'"Carry the digit from the tens place.\n###\n15 // 10 = 1\n###\n{x} // 10 =",{x // 10}\n')

In [4]:
with open('summation.csv', 'w') as file:
  for x in range(0, 1000):
    for y in range(0, 1000):
      file.write(f'"Add two numbers.\n###\n5 + 3 = 8\n###\n{x} + {y} =",{x + y}\n')

In [5]:
with open('concatenation.csv', 'w') as file:
  for x in range(0, 100):
    for y in range(0, 100):
      for z in range(0, 100):
        file.write(f'"Concatenate the numbers.\n###\n1, 5, 3 = 153\n###\n{x}, {y}, {z} =",{x}{y}{z}\n')

In [6]:
!pip install accelerate
!pip install transformers
!pip install sentencepiece
!pip install accelerate

In [7]:
from datasets import load_dataset

multiply_datasets = load_dataset("csv", data_files="multiply.csv", delimiter=',', column_names=['question', 'answer'])["train"].train_test_split(test_size=0.1)
multiply_1_digit_datasets = load_dataset("csv", data_files="multiply_1_digit.csv", delimiter=',', column_names=['question', 'answer'])["train"].train_test_split(test_size=0.1)
carry_datasets = load_dataset("csv", data_files="carry.csv", delimiter=',', column_names=['question', 'answer'])["train"].train_test_split(test_size=0.1)
summation_datasets = load_dataset("csv", data_files="summation.csv", delimiter=',', column_names=['question', 'answer'])["train"].train_test_split(test_size=0.1)
concatenation_datasets = load_dataset("csv", data_files="concatenation.csv", delimiter=',', column_names=['question', 'answer'])["train"].train_test_split(test_size=0.1)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [8]:
multiply_datasets

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 900000
    })
    test: Dataset({
        features: ['question', 'answer'],
        num_rows: 100000
    })
})

In [9]:
from transformers import DataCollatorWithPadding
from transformers import T5Tokenizer

tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")

def tokenize_function(examples):
    return tokenizer(examples["question"], text_target=list(map(str, examples["answer"])), padding="max_length", truncation=True, return_tensors="pt")

tokenized_multiply_datasets = multiply_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
tokenized_multiply_datasets = tokenized_multiply_datasets.remove_columns(["question", "answer"])
tokenized_multiply_datasets

Map:   0%|          | 0/900000 [00:00<?, ? examples/s]

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 900000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 100000
    })
})

In [22]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_multiply_datasets["train"], shuffle=True, batch_size=1, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    tokenized_multiply_datasets["test"], batch_size=1, collate_fn=data_collator
)

In [23]:
for batch in train_dataloader:
    break
{k: v.shape for k, v in batch.items()}

{'input_ids': torch.Size([1, 512]),
 'attention_mask': torch.Size([1, 512]),
 'labels': torch.Size([1, 512])}

In [24]:
print(torch.cuda.memory_summary(device=None, abbreviated=False)) 

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 4            |        cudaMalloc retries: 4         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |   5400 MiB |   5400 MiB |   5592 MiB | 196864 KiB |
|       from large pool |   5397 MiB |   5397 MiB |   5586 MiB | 192768 KiB |
|       from small pool |      2 MiB |      4 MiB |      6 MiB |   4096 KiB |
|---------------------------------------------------------------------------|
| Active memory         |   5400 MiB |   5400 MiB |   5592 MiB | 196864 KiB |
|       from large pool |   5397 MiB |   5397 MiB |   5586 MiB |

In [27]:
from transformers import T5ForConditionalGeneration, get_scheduler
from torch.optim import AdamW
import torch
from tqdm.auto import tqdm
import evaluate

model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base")
optimizer = AdamW(model.parameters(), lr=3e-5)
metric = evaluate.load("glue", "mrpc")

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_epochs):
    model.train()
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)
        
    model.eval()
    for batch in eval_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)

        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        metric.add_batch(predictions=predictions, references=batch["labels"])

    metric.compute()

OutOfMemoryError: CUDA out of memory. Tried to allocate 96.00 MiB (GPU 0; 6.00 GiB total capacity; 5.27 GiB already allocated; 0 bytes free; 5.35 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
# from transformers import T5Tokenizer, T5ForConditionalGeneration

# input_ids = tokenizer("translate English to German: The house is wonderful.", return_tensors="pt").input_ids
# labels = tokenizer("Das Haus ist wunderbar.", return_tensors="pt").input_ids

# # the forward function automatically creates the correct decoder_input_ids
# outputs = model(**batch)
# print(outputs.loss, outputs.logits.shape)

# loss = model(input_ids=input_ids, labels=labels).loss
# loss.item()

In [ ]:
# from transformers import T5Tokenizer, T5Model

# tokenizer = T5Tokenizer.from_pretrained("t5-base")
# model = T5Model.from_pretrained("t5-base")

# # input_ids = tokenizer(
# #     padding="max_length",
# #     truncation=True,
# #     model_max_length=int(1e30), return_tensors="pt"
# # ).input_ids  # Batch size 1
# decoder_input_ids = tokenizer("", return_tensors="pt").input_ids  # Batch size 1

# # forward pass
# outputs = model(input_ids=input_ids, decoder_input_ids=decoder_input_ids)
# last_hidden_states = outputs.last_hidden_state

ValueError: text input must of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).